In [ ]:
# Obrigatório para testes no Google Colab, senão ignorar
from google.colab import userdata

github_token = userdata.get('GITHUB_TOKEN')
!git clone https://{github_token}@github.com/erickrribeiro/ufam_nlp_01_2025.git
!mv ./ufam_nlp_01_2025/scripts/** ./
!cp ./ufam_nlp_01_2025/models/* ./
!unzip ./mistral-sql-lora-v3.zip -d ./
!unzip ./mistral-sql-lora-v4.zip -d ./

In [ ]:
# !rm -rf ./ufam_nlp_01_2025
# !rm -rf ./spider
# !rm -rf mistral-sql-lora-*
# !rm *.py
# !rm *.ipynb
# !rm *.csv
# !rm *.zip

In [ ]:
from utils import download_spider_dataset

download_spider_dataset()

In [ ]:
!unzip spider_data.zip -d spider

In [ ]:
!pip install -U gdown==5.2.0 \
                transformers==4.52.4 \
                trl==0.18.2 \
                peft==0.15.2 \
                datasets==3.6.0 \
                accelerate==1.8.0 \
                huggingface_hub==0.33.0 \
                fsspec==2025.3.0 \
                deepeval==3.1.6 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.6/558.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import datasets, transformers, trl, peft, accelerate, huggingface_hub, fsspec, deepeval

print("datasets:", datasets.__version__)
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("trl:", trl.__version__)
print("accelerate:", accelerate.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("fsspec:", fsspec.__version__)
print("deepeval:", deepeval.__version__)

datasets: 3.6.0
transformers: 4.52.4
peft: 0.15.2
trl: 0.18.2
accelerate: 1.8.0
huggingface_hub: 0.33.0
fsspec: 2025.3.0
deepeval: 3.1.6


In [ ]:
from utils import (
  extract_sql,
  generate_schema_description,
  load_tables,
  load_few_shot,
  set_seed,
  evaluate,
)

tables = load_tables("./spider/spider_data/tables.json")
schema_cache = {
  t["db_id"]: generate_schema_description(t) for t in tables
}
few_shot = load_few_shot()

- Funções úteis

In [1]:
import json
import re
import random
import numpy as np
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # Garante determinismo
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def load_tables(file_path):
  with open(file_path, "r") as f:
    return json.load(f)

def build_prompts_batch(examples):
  inputs = []
  labels = []

  for db_id, question, sql in zip(examples["db_id"], examples["question"], examples["query"]):
    schema = schema_cache[db_id]

    chat = [{"role": "system", "content": SYSTEM_PROMPT}]

    # Few-shot learning
    for example in few_shot:
      schema_i = schema_cache[example["db_id"]]
      chat.append({
        "role": "user",
        "content": USER_PROMPT_TEMPLATE.format(schema_i, example["question"]),
      })
      sql = example["query"]
      chat.append({
        "role": "assistant",
        "content": f"```sql {sql} ```",
      })

    # Current example
    chat.append({
      "role": "user",
      "content": USER_PROMPT_TEMPLATE.format(schema, question),
    })

    prompt = tokenizer.apply_chat_template(
      chat, tokenize=False, add_generation_prompt=True
    )

    target = f"```sql {sql} ```"

    # Tokeniza separadamente o prompt e o target
    prompt_ids = tokenizer(
      prompt,
      padding="max_length",
      truncation=True,
      max_length=1024,
    )

    target_ids = tokenizer(
      target,
      padding="max_length",
      truncation=True,
      max_length=1024,
    )

    input_ids = prompt_ids["input_ids"]
    attention_mask = prompt_ids["attention_mask"]
    labels = target_ids["input_ids"]

    # Troca padding tokens por -100 nos labels para não afetar o cálculo da loss
    labels = [
      (label if label != tokenizer.pad_token_id else -100)
      for label in labels
    ]

    inputs.append({
      "prompt": prompt,
    })

  # Organiza o batch no formato esperado pelo Trainer
  batch = { key: [dic[key] for dic in inputs] for key in inputs[0] }

  return batch

def extract_sql(response):
  # Procura dentro de blocos ```sql``` ... ```
  match = re.search(r"```sql(.*?)```", response, re.DOTALL | re.IGNORECASE)

  if match:
      sql = match.group(1).strip()
      return sql
  else:
      # Se não tiver bloco ```sql```, tenta pegar a primeira query SELECT na resposta
      fallback = re.search(r"(SELECT .*?;)", response, re.DOTALL | re.IGNORECASE)
      if fallback:
          return fallback.group(1).strip()
      else:
          return response

def generate_schema_description(table):
    table_names = table["table_names_original"]
    columns = table["column_names_original"]
    foreign_keys = table["foreign_keys"]

    # Mapeia tabelas para suas colunas
    table_to_columns = {name: [] for name in table_names}

    for col in columns:
        table_idx, col_name = col
        if table_idx == -1:
            continue  # Ignora o '*'
        table_name = table_names[table_idx]
        table_to_columns[table_name].append(col_name)

    # Monta descrição das tabelas
    table_descriptions = []
    for table_name, cols in table_to_columns.items():
        col_list = " , ".join(cols)
        table_descriptions.append(f"| {table_name} : {col_list} |")

    # Monta descrição das foreign keys
    fk_descriptions = []
    for fk in foreign_keys:
        col1_idx, col2_idx = fk

        table1_idx, col1 = columns[col1_idx]
        table2_idx, col2 = columns[col2_idx]

        table1 = table_names[table1_idx]
        table2 = table_names[table2_idx]

        fk_descriptions.append(
            f"{table1}.{col1} = {table2}.{col2}"
        )

    # Juntar tudo
    description = "\n".join(table_descriptions)

    if fk_descriptions:
        description += "\nRelationships:\n" + " , ".join(fk_descriptions)

    return description


SYSTEM_PROMPT = """ You are a helpful assistant who answers questions about database tables
by responding with SQL queries."""

USER_PROMPT_TEMPLATE = """Schema:
{0}

Question: {1}
SQL:
"""
def build_prompt(
    schema: str,
    question: str,
    few_shot: list[dict] = [],
    system_prompt: str = SYSTEM_PROMPT,
) -> str:
    tables = load_tables("tables.json")
    chat = [
      {
        "role": "system",
        "content": SYSTEM_PROMPT,
      },
    ]
    # Few-shot learning
    for example in few_shot:
      table = next(t for t in tables if t["db_id"] == example["db_id"])
      schema_i = generate_schema_description(table)
      chat.append({
        "role": "user",
        "content": USER_PROMPT_TEMPLATE.format(schema_i, example["question"]),
      })
      sql = example["query"]
      chat.append({
        "role": "assistant",
        "content": f"```sql {sql} ```",
      })

    chat.append({
      "role": "user",
      "content": USER_PROMPT_TEMPLATE.format(schema, question),
    })

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    return prompt

# Fase 1: Estabelecimento do Baseline de Desempenho


## 1.1. Prompt Engineering:

A seguir, é definir um prompt com few-shot para a tarefa `Text-to-SQL`. O prompt deve contem 3 exemplos representativos extraídos do training split do Spider. Este template de prompt será utilizado em todas as avaliações de baseline.


In [ ]:
import torch
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=userdata.get('HF_TOKEN'),
                                          model_max_length=1024)
model = AutoModelForCausalLM.from_pretrained(model_name, token=userdata.get('HF_TOKEN'))
model.to(device)

print("Available device: ", device)
print("Model device: ", model.device)

In [ ]:
schema = """| department : Department_ID , Name , Creation , Ranking , Budget_in_Billions , Num_Employees |
| head : head_ID , name , born_state , age |
| management : department_ID , head_ID , temporary_acting |
Relationships:
management.head_ID = head.head_ID , management.department_ID = department.Department_ID
"""

question = ""
prompt = build_prompt(schema, question)
input = tokenizer(prompt, return_tensors="pt").to(device)

# Remove o prompt da saída
input_ids = input["input_ids"]
with torch.no_grad():
  output = model.generate(**input, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
  generated_tokens = output[0][input_ids.shape[1]:]

# output = model.generate(**input, max_new_tokens=256, do_sample=True, temperature=0.01)
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(response)

SELECT COUNT(*) FROM head
WHERE age > 56;


## 1.2 Execução da Avaliação

A seguir, o modelo base `mistralai/Mistral-7B-Instruct-v0.2` (não treinado) será submetido ao Spider dev split, utilizando o template de prompt definido na função `build_prompt`.

In [ ]:
prompt = build_prompt(schema="fake_schema", question="fake_question", few_shot=[
  {"db_id": ds["train"][1]["db_id"], "question": ds["train"][1]["question"], "query": ds["train"][1]["query"]},
  {"db_id": ds["train"][20]["db_id"], "question": ds["train"][20]["question"], "query": ds["train"][20]["query"]},
  {"db_id": ds["train"][65]["db_id"], "question": ds["train"][65]["question"], "query": ds["train"][65]["query"]},
])

print(prompt)

<s> [INST]  You are a helpful assistant who answers questions about database tables 
by responding with SQL queries.

Schema: 
| department : Department_ID , Name , Creation , Ranking , Budget_in_Billions , Num_Employees |
| head : head_ID , name , born_state , age |
| management : department_ID , head_ID , temporary_acting |
Relationships:
management.head_ID = head.head_ID , management.department_ID = department.Department_ID

Question: List the name, born state and age of the heads of departments ordered by age.
SQL:
 [/INST] ```sql SELECT name ,  born_state ,  age FROM head ORDER BY age ```</s> [INST] Schema: 
| city : City_ID , Official_Name , Status , Area_km_2 , Population , Census_Ranking |
| farm : Farm_ID , Year , Total_Horses , Working_Horses , Total_Cattle , Oxen , Bulls , Cows , Pigs , Sheep_and_Goats |
| farm_competition : Competition_ID , Year , Theme , Host_city_ID , Hosts |
| competition_record : Competition_ID , Farm_ID , Rank |
Relationships:
farm_competition.Host_cit

In [ ]:
from tqdm.notebook import tqdm

tables = load_tables("tables.json")
results = []
i = 1
for row in tqdm(ds["validation"]):
  db_id = row["db_id"]
  question = row["question"]
  gold_sql = row["query"]

  table = next(t for t in tables if t["db_id"] == db_id)
  schema_text = generate_schema_description(table)
  prompt = build_prompt(schema_text, question)

  input = tokenizer(prompt, return_tensors="pt").to(device)
  input_ids = input["input_ids"]
  with torch.no_grad():
    output = model.generate(
      **input,
      max_new_tokens=100,
      pad_token_id=tokenizer.eos_token_id)
  generated_tokens = output[0][input_ids.shape[1]:]
  response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
  if i % 100 ==0:
    print("question:", question)
    print("gold_sql:", gold_sql)
    print("predicted_sql:", response)

  results.append({
    "db_id": db_id,
    "question": question,
    "gold_sql": gold_sql,
    "predicted_sql": extract_sql(response)
  })
  i+= 1

  0%|          | 0/1034 [00:00<?, ?it/s]

question: Find the name of the makers that produced some cars in the year of 1970?
gold_sql: SELECT DISTINCT T1.Maker FROM CAR_MAKERS AS T1 JOIN MODEL_LIST AS T2 ON T1.Id  =  T2.Maker JOIN CAR_NAMES AS T3 ON T2.model  =  T3.model JOIN CARS_DATA AS T4 ON T3.MakeId  =  T4.id WHERE T4.year  =  '1970';
predicted_sql: SELECT car_makers.FullName
FROM car_makers
JOIN car_names ON car_makers.Id = car_names.Maker
JOIN cars_data ON car_names.Model = cars_data.MakeId
WHERE cars_data.Year = 1970;
question: What is the airport name for airport 'AKO'?
gold_sql: SELECT AirportName FROM AIRPORTS WHERE AirportCode  =  "AKO"
predicted_sql: SELECT AirportName FROM airports WHERE AirportCode = 'AKO';
question: List document IDs, document names, and document descriptions for all documents.
gold_sql: SELECT document_id ,  document_name ,  document_description FROM Documents
predicted_sql: SELECT Documents.Document_ID, Documents.Document_Name, Documents.Document_Description
FROM Documents;
question: Show nam

Token indices sequence length is longer than the specified maximum sequence length for this model (623 > 512). Running this sequence through the model will result in indexing errors


question: What is the content of TV Channel with serial name "Sky Radio"?
gold_sql: SELECT Content FROM TV_Channel WHERE series_name = "Sky Radio";
predicted_sql: SELECT Content
FROM TV_Channel
WHERE series_name = 'Sky Radio';
question: What are the create dates, states, and phone numbers of the votes that were for the contestant named 'Tabatha Gehling'?
gold_sql: SELECT T2.created ,  T2.state ,  T2.phone_number FROM contestants AS T1 JOIN votes AS T2 ON T1.contestant_number  =  T2.contestant_number WHERE T1.contestant_name  =  'Tabatha Gehling'
predicted_sql: SELECT VOTES.created, AREA_CODE_STATE.state, VOTES.phone_number
FROM VOTES
INNER JOIN CONTESTANTS ON VOTES.contestant_number = CONTESTANTS.contestant_number
INNER JOIN AREA_CODE_STATE ON VOTES.state = AREA_CODE_STATE.state
WHERE CONTESTANTS.contestant_name = '
question: What are the different continents and the total popuation and average life expectancy corresponding to each, for continents that have an average life expectancy l

In [ ]:
results[:5]

[{'db_id': 'concert_singer',
  'question': 'How many singers do we have?',
  'gold_sql': 'SELECT count(*) FROM singer',
  'predicted_sql': 'SELECT COUNT(*) FROM singer;'},
 {'db_id': 'concert_singer',
  'question': 'What is the total number of singers?',
  'gold_sql': 'SELECT count(*) FROM singer',
  'predicted_sql': 'SELECT COUNT(*) FROM singer;'},
 {'db_id': 'concert_singer',
  'question': 'Show name, country, age for all singers ordered by age from the oldest to the youngest.',
  'gold_sql': 'SELECT name ,  country ,  age FROM singer ORDER BY age DESC',
  'predicted_sql': 'SELECT Name, Country, Age\nFROM singer\nORDER BY Age DESC;'},
 {'db_id': 'concert_singer',
  'question': 'What are the names, countries, and ages for every singer in descending order of age?',
  'gold_sql': 'SELECT name ,  country ,  age FROM singer ORDER BY age DESC',
  'predicted_sql': 'SELECT singer.Name, singer.Country, singer.Age\nFROM singer\nORDER BY singer.Age DESC;'},
 {'db_id': 'concert_singer',
  'quest

## 1.3. Coleta de Dados

A seguir, será registrado o resultado da consulta SQL gerada para cada entrada utilizando o modelo `mistralai/Mistral-7B-Instruct-v0.2`. A métrica a ser reportada nesta fase é a contagem bruta de sucesso/falha.


In [ ]:
import sqlite3

def execute_and_fetch(db_path, query):
  try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(query)
    results = cursor.fetchall()

    conn.close()

    # Ordena os resultados para comparação
    results_sorted = sorted(results)

    return True, results_sorted, None
  except Exception as e:
      return False, None, str(e)


In [ ]:
import os

execution_match = 0
total_evaluated = 0

for row in tqdm(results):
    db_id = row["db_id"]
    db_path = os.path.join("spider/spider_data/database", db_id, f"{db_id}.sqlite")

    if not os.path.exists(db_path):
        row["execution_accuracy"] = "DB_NOT_FOUND"
        continue

    gold_query = row["gold_sql"]
    predicted_query = row["predicted_sql"]

    # Executa gold
    gold_success, gold_result, gold_error = execute_and_fetch(db_path, gold_query)

    # Executa predicted
    pred_success, pred_result, pred_error = execute_and_fetch(db_path, predicted_query)

    # Avaliação
    if gold_success and pred_success:
        total_evaluated += 1

        if gold_result == pred_result:
            execution_match += 1
            row["execution_accuracy"] = "MATCH"
        else:
            row["execution_accuracy"] = "MISMATCH"
    else:
        error_msg = ""
        if not gold_success:
            error_msg += f"GOLD_FAIL: {gold_error} | "
        if not pred_success:
            error_msg += f"PRED_FAIL: {pred_error}"
        row["execution_accuracy"] = error_msg.strip()

print(f"Total comparado (com sucesso em ambas): {total_evaluated}")
print(f"✔️ Matches: {execution_match}")
print(f"❌ Mismatches: {total_evaluated - execution_match}")
print(f"✅ Execution Accuracy: {execution_match / total_evaluated * 100:.2f}%")

  0%|          | 0/1034 [00:00<?, ?it/s]

Total comparado (com sucesso em ambas): 662
✔️ Matches: 433
❌ Mismatches: 229
✅ Execution Accuracy: 65.41%


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv("spider_execution_accuracy_results.csv", index=False)

# Fase 2: Execução do Fine-Tuning

## 2.1. PEFT (Parameter-Efficient Fine-Tuning):

A seguir, a implementação do fine-tuning utilizando a técnica LoRA (Low-Rank Adaptation) no modelo `mistralai/Mistral-7B-Instruct-v0.2`.


Para o fine-tuning vou carregar o modelo usando `bfloat16`. Pois, intervalo dinâmico é igual ao fp32, apenas com menos bits na mantissa (menos precisão na casa decimal). Muito mais estável numericamente que fp16, mas com mesma aceleração via Tensor Cores. Além de ser suportado apenas na GPUs A100 que estou utilizando.

In [ ]:
import torch
from google.colab import userdata
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=userdata.get('HF_TOKEN'),
                                          model_max_length=1024)
# https://github.com/huggingface/transformers/issues/29936?utm_source=chatgpt.com
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map="auto",
  torch_dtype=torch.float16,
  token=userdata.get('HF_TOKEN'))


lora_config = LoraConfig(
  r=4,                          # rank da LoRA (ajustável)
  lora_alpha=16,                # fator de escalonamento
  lora_dropout=0.05,            # probabilidade de dropout nas LoRAs
  bias="none",
  task_type=TaskType.CAUSAL_LM, # modelo auto-regressivo
  target_modules=["q_proj", "v_proj"]  # módulos alvo — atenção
)

print("Available device: ", device)
print("Model device: ", base_model.device)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Available device:  cuda
Model device:  cuda:0
trainable params: 1,703,936 || all params: 7,243,436,032 || trainable%: 0.0235


In [7]:
from datasets import load_dataset

ds = load_dataset("xlangai/spider")
train_dataset = ds["train"]
eval_dataset = ds["validation"]

# Pré-carrega os schemas no formato texto
tables = load_tables("./spider/spider_data/tables.json")
schema_cache = {
  t["db_id"]: generate_schema_description(t) for t in tables
}
few_shot=[
  {"db_id": ds["train"][1]["db_id"], "question": ds["train"][1]["question"], "query": ds["train"][1]["query"]},
  {"db_id": ds["train"][20]["db_id"], "question": ds["train"][20]["question"], "query": ds["train"][20]["query"]},
  {"db_id": ds["train"][65]["db_id"], "question": ds["train"][65]["question"], "query": ds["train"][65]["query"]},
]
few_shot

[{'db_id': 'department_management',
  'question': 'List the name, born state and age of the heads of departments ordered by age.',
  'query': 'SELECT name ,  born_state ,  age FROM head ORDER BY age'},
 {'db_id': 'farm',
  'question': 'What are the hosts of competitions whose theme is not "Aliens"?',
  'query': "SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens'"},
 {'db_id': 'student_assessment',
  'question': 'What are the ids of all students who have attended at least one course?',
  'query': 'SELECT student_id FROM student_course_attendance'}]

In [9]:
def preprocess_function(examples):
  inputs = []
  labels = []

  for db_id, question, sql in zip(examples["db_id"], examples["question"], examples["query"]):
    schema = schema_cache[db_id]

    chat = [{"role": "system", "content": SYSTEM_PROMPT}]

    # Few-shot learning
    for example in few_shot:
      schema_i = schema_cache[example["db_id"]]
      chat.append({
        "role": "user",
        "content": USER_PROMPT_TEMPLATE.format(schema_i, example["question"]),
      })
      sql = example["query"]
      chat.append({
        "role": "assistant",
        "content": f"```sql {sql} ```",
      })

    # Current example
    chat.append({
      "role": "user",
      "content": USER_PROMPT_TEMPLATE.format(schema, question),
    })

    prompt = tokenizer.apply_chat_template(
      chat, tokenize=False, add_generation_prompt=True
    )

    target = f"```sql {sql} ```"

    # Tokeniza separadamente o prompt e o target
    prompt_ids = tokenizer(
      prompt,
      padding="max_length",
      truncation=True,
      max_length=1024,
    )

    target_ids = tokenizer(
      target,
      padding="max_length",
      truncation=True,
      max_length=1024,
    )

    input_ids = prompt_ids["input_ids"]
    attention_mask = prompt_ids["attention_mask"]
    labels = target_ids["input_ids"]

    # Troca padding tokens por -100 nos labels para não afetar o cálculo da loss
    labels = [
      (label if label != tokenizer.pad_token_id else -100)
      for label in labels
    ]

    inputs.append({
      "prompt": prompt,
      "target": target,
      "input_ids": input_ids,
      "attention_mask": attention_mask,
      "labels": labels,
    })

  # Organiza o batch no formato esperado pelo Trainer
  batch = { key: [dic[key] for dic in inputs] for key in inputs[0] }

  return batch


- Tokenização do dataset

In [ ]:
tokenized_train = train_dataset.map(
  preprocess_function,
  batched=True,
  remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
  preprocess_function,
  batched=True,
  remove_columns=eval_dataset.column_names
)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7000
})

- Configuração do Treinamento

| Otimizador          | Backend         | Velocidade | Memória         | Observações                                       |
| ------------------- | --------------- | ---------- | --------------- | ------------------------------------------------- |
| `adamw_hf`          | Hugging Face    | Média      | Média           | Padrão, robusto                                   |
| `adamw_torch`       | PyTorch         | Boa        | Boa             | Sem fused                                         |
| `adamw_torch_fused` | PyTorch (fused) | **Alta**   | **Baixa**       | Recomendo se seu hardware suportar                |
| `adamw_apex_fused`  | NVIDIA Apex     | Alta       | Média           | Depende do Apex, menos portátil                   |
| `adamw_bnb_8bit`    | bitsandbytes    | Variável   | **Muito baixa** | Otimização extrema de memória (quantização 8-bit) |


## 2.3. Experimentação de Hiperparâmetros

A seguir é testado dois hiperparâmetros de treinamento. Sugestão: varie a taxa de aprendizado (learning_rate) ou o número de épocas (num_train_epochs). O objetivo é observar o impacto dessas variações no resultado final.


## Experimento 1
- Learning rate: 2e-4
- Batch size: 4
- Épocas: 2
- LoRA:
  - r=4
  - alpha=16
  - dropout=0.05
  - target_modules=["q_proj", "v_proj"]

> Nota: é uma configuração do LoRA e Batch size foi ajustada para caber na memória da GPU A100 (40GB). O uso uso de Learning rate mais conservador tem como objetibo tornar o treinamento mais estável.

In [ ]:
import transformers
from trl import SFTTrainer

batch_size = 4
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=True,
        output_dir="outputs",
        logging_dir="./logs",
        logging_strategy="steps",
        logging_steps=100,
        optim="adamw_torch_fused",
        report_to="none"
    ),
    peft_config=lora_config,
)

trainer.train()

Truncating train dataset:   0%|          | 0/7000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.325100
200,0.057100
300,0.035900
400,0.029400
500,0.022700
600,0.020300
700,0.019100
800,0.019300


TrainOutput(global_step=876, training_loss=0.06192664645578219, metrics={'train_runtime': 2836.5407, 'train_samples_per_second': 4.936, 'train_steps_per_second': 0.309, 'total_flos': 6.11777104576512e+17, 'train_loss': 0.06192664645578219})

In [ ]:
model.save_pretrained("./mistral-sql-lora-v3", save_adapter=True, save_config=True)

In [ ]:
!cp -r /content/mistral-sql-lora-v3 /content/drive/MyDrive/PPGI/Doutorado/Disciplinas/NLP/nlp-2025-tp-04

In [9]:
!cp -r /content/drive/MyDrive/PPGI/Doutorado/Disciplinas/NLP/nlp-2025-tp-04/mistral-sql-lora-v2 /content/mistral-sql-lora-v2

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel, PeftConfig

lora_path = "./mistral-sql-lora-v3"
config = PeftConfig.from_pretrained(lora_path)
base_model = AutoModelForCausalLM.from_pretrained(
  config.base_model_name_or_path,
  device_map="auto",
  torch_dtype="auto"
)
peft_model_v3 = PeftModel.from_pretrained(base_model, lora_path)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Pipeline
pipe_v3 = pipeline(
  "text-generation",
  model=peft_model_v3,
  tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
from tqdm.notebook import tqdm
from datasets import load_dataset

dataset_eval = load_dataset('csv', data_files='dataset_eval.csv')
prompts = dataset_eval["train"]['prompt']

from tqdm.notebook import tqdm

outputs = []
batch_size = 8

for i in tqdm(range(0, len(prompts), batch_size)):
  batch = prompts[i:i+batch_size]
  out = pipe_v3(
      batch,
      max_new_tokens=200,
      pad_token_id=tokenizer.eos_token_id
  )
  outputs.extend(out)
len(outputs)

  0%|          | 0/130 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [22]:
import pandas as pd

results = []
for idx, output in enumerate(outputs):
  db_id = dataset_eval["train"][idx]["db_id"]
  question = dataset_eval["train"][idx]["question"]
  gold_sql = dataset_eval["train"][idx]["query"]

  offset = len(dataset_eval["train"][idx]["prompt"])
  predicted_sql = extract_sql(output[0]["generated_text"][offset:])

  results.append({
    "db_id": db_id,
    "question": question,
    "gold_sql": gold_sql,
    "predicted_sql": predicted_sql
  })

df = pd.DataFrame(results)
df.to_csv("./mistral-sql-lora-v3-predictions.csv", index=False)


In [12]:
from datasets import load_dataset

ds = load_dataset("xlangai/spider")
train_dataset = ds["train"]
eval_dataset = ds["validation"]

# Pré-carrega os schemas no formato texto
tables = load_tables("./spider/spider_data/tables.json")
schema_cache = {
  t["db_id"]: generate_schema_description(t) for t in tables
}
few_shot=[
  {"db_id": ds["train"][1]["db_id"], "question": ds["train"][1]["question"], "query": ds["train"][1]["query"]},
  {"db_id": ds["train"][20]["db_id"], "question": ds["train"][20]["question"], "query": ds["train"][20]["query"]},
  {"db_id": ds["train"][65]["db_id"], "question": ds["train"][65]["question"], "query": ds["train"][65]["query"]},
]

prompts_eval = eval_dataset.map(
  build_prompts_batch,
  batched=True,
)
prompts_eval.to_csv("./dataset_eval.csv")

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [19]:
!cp -r /content/dataset_eval.csv /content/drive/MyDrive/PPGI/Doutorado/Disciplinas/NLP/nlp-2025-tp-04

## Experimento 2
- Learning rate: 5e-4
- Batch size: 4
- Épocas: 2
- LoRA:
  - r=4
  - alpha=16
  - dropout=0.05
  - target_modules=["q_proj", "v_proj"]

> Nota: é uma configuração do LoRA e Batch size ajustada para caber na memória da GPU A100 (40GB). Já o uso de Learning rate mais agressivo  tem como objetivo verificar se o aumento da learning rate gera instabilidade ou overfitting.

In [ ]:
import transformers
from trl import SFTTrainer

batch_size = 4
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        num_train_epochs=2,
        learning_rate=5e-4,
        fp16=True,
        output_dir="outputs",
        logging_dir="./logs",
        logging_strategy="steps",
        logging_steps=100,
        optim="adamw_torch_fused",
        report_to="none"
    ),
    peft_config=lora_config,
)

trainer.train()

Truncating train dataset:   0%|          | 0/7000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,0.231300
200,0.040500
300,0.023700
400,0.020700
500,0.019000
600,0.018700
700,0.017800
800,0.018000


TrainOutput(global_step=876, training_loss=0.045906893738872925, metrics={'train_runtime': 2835.1315, 'train_samples_per_second': 4.938, 'train_steps_per_second': 0.309, 'total_flos': 6.11777104576512e+17, 'train_loss': 0.045906893738872925})

In [ ]:
model.save_pretrained("./mistral-sql-lora-v4", save_adapter=True, save_config=True)

In [ ]:
!cp -r /content/mistral-sql-lora-v4 /content/drive/MyDrive/PPGI/Doutorado/Disciplinas/NLP/nlp-2025-tp-04

## (Debug) Teste do modelo base + adaptador LoRA

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

device = "cuda" if torch.cuda.is_available() else "cpu"
lora_path = "./mistral-sql-lora-v2" #epoch 1 lr=2e-4
# lora_path = "./mistral-sql-lora-v3" #epoch 2 lr=2e-4
# lora_path = "./mistral-sql-lora-v4" #epoch 2 lr=5e-4

config = PeftConfig.from_pretrained(lora_path)
base_model = AutoModelForCausalLM.from_pretrained(
  config.base_model_name_or_path,
  device_map="auto",
  torch_dtype="auto",
).to(device)

peft_model = PeftModel.from_pretrained(base_model, lora_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, model_max_length=1024)
merged_model = peft_model.merge_and_unload()



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

ds = load_dataset("xlangai/spider")
train_dataset = ds["train"]
eval_dataset = ds["validation"]

# Pré-carrega os schemas no formato texto
tables = load_tables("./spider/spider_data/tables.json")
schema_cache = {
  t["db_id"]: generate_schema_description(t) for t in tables
}
few_shot=[
  {"db_id": ds["train"][1]["db_id"], "question": ds["train"][1]["question"], "query": ds["train"][1]["query"]},
  {"db_id": ds["train"][20]["db_id"], "question": ds["train"][20]["question"], "query": ds["train"][20]["query"]},
  {"db_id": ds["train"][65]["db_id"], "question": ds["train"][65]["question"], "query": ds["train"][65]["query"]},
]
few_shot

[{'db_id': 'department_management',
  'question': 'List the name, born state and age of the heads of departments ordered by age.',
  'query': 'SELECT name ,  born_state ,  age FROM head ORDER BY age'},
 {'db_id': 'farm',
  'question': 'What are the hosts of competitions whose theme is not "Aliens"?',
  'query': "SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens'"},
 {'db_id': 'student_assessment',
  'question': 'What are the ids of all students who have attended at least one course?',
  'query': 'SELECT student_id FROM student_course_attendance'}]

In [ ]:
# Escolhe um exemplo para teste
n = 4
db_id = ds["validation"][n]["db_id"]
question = ds["validation"][n]["question"]
gold_sql = ds["validation"][n]["query"]

# Pré-carrega os schemas no formato texto
tables = load_tables("./spider/spider_data/tables.json")
schema_cache = {
  t["db_id"]: generate_schema_description(t) for t in tables
}
# Carrega os exemplos de few_shot
few_shot=[
  {"db_id": ds["train"][1]["db_id"], "question": ds["train"][1]["question"], "query": ds["train"][1]["query"]},
  {"db_id": ds["train"][20]["db_id"], "question": ds["train"][20]["question"], "query": ds["train"][20]["query"]},
  {"db_id": ds["train"][65]["db_id"], "question": ds["train"][65]["question"], "query": ds["train"][65]["query"]},
]

schema = schema_cache[db_id]

chat = [{"role": "system", "content": SYSTEM_PROMPT}]
# Few-shot learning
for example in few_shot:
  schema_i = schema_cache[example["db_id"]]
  chat.append({
    "role": "user",
    "content": USER_PROMPT_TEMPLATE.format(schema_i, example["question"]),
  })
  sql = example["query"]
  chat.append({
    "role": "assistant",
    "content": f"```sql {sql} ```",
  })

# Current example
chat.append({
  "role": "user",
  "content": USER_PROMPT_TEMPLATE.format(schema, question),
})

prompt = tokenizer.apply_chat_template(
  chat, tokenize=False, add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]
outputs = merged_model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
generated_tokens = outputs[0][input_ids.shape[1]:]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
predicted_sql = extract_sql(response)

print("Question:", question)
print("gold_sql:", gold_sql)
print("predicted_sql:", predicted_sql)

Token indices sequence length is longer than the specified maximum sequence length for this model (1030 > 1024). Running this sequence through the model will result in indexing errors


Question: What is the average, minimum, and maximum age of all singers from France?
gold_sql: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
predicted_sql: SELECT AVG(Age) , MIN(Age) , MAX(Age) FROM singer WHERE Country =  'France'


In [ ]:
!cp -r /content/mistral-sql-lora-v3 /content/drive/MyDrive/PPGI/Doutorado/Disciplinas/NLP/nlp-2025-tp-04

# Fase 3: Avaliação de Desempenho na Tarefa-Alvo com Métrica Customizada


## 3.1. Implementação da Métrica:

A seguri será desenvolvida uma métrica customizada em DeepEval para "Execution Accuracy". A classe da métrica deve seguir a seguinte estrutura:

- Herdar de deepeval.metrics.BaseMetric.
- Implementar o método measure(self, test_case: LLMTestCase) -> float.
- A lógica interna do measure deve:
    - a. Estabelecer uma conexão com um banco de dados sqlite contendo a base de dados de teste do Spider.
    - b. Executar a consulta SQL contida em test_case.actual_outputdentro de uma transação segura (para lidar com erros de sintaxe via try-except).
    - c. Executar a consulta ground truthcontida em test_case.expected_output.
    - d. Comparar os conjuntos de resultados. A comparação deve ser insensível à ordem das linhas. e. Retornar 1.0 para sucesso (resultados idênticos) e 0.0 para falha.

In [ ]:
import os

execution_match = 0
total_evaluated = 0

for row in tqdm(results):
    db_id = row["db_id"]
    db_path = os.path.join("spider/spider_data/database", db_id, f"{db_id}.sqlite")

    if not os.path.exists(db_path):
        row["execution_accuracy"] = "DB_NOT_FOUND"
        continue

    gold_query = row["gold_sql"]
    predicted_query = row["predicted_sql"]

    # Executa gold
    gold_success, gold_result, gold_error = execute_and_fetch(db_path, gold_query)

    # Executa predicted
    pred_success, pred_result, pred_error = execute_and_fetch(db_path, predicted_query)

    # Avaliação
    if gold_success and pred_success:
        total_evaluated += 1

        if gold_result == pred_result:
            execution_match += 1
            row["execution_accuracy"] = "MATCH"
        else:
            row["execution_accuracy"] = "MISMATCH"
    else:
        error_msg = ""
        if not gold_success:
            error_msg += f"GOLD_FAIL: {gold_error} | "
        if not pred_success:
            error_msg += f"PRED_FAIL: {pred_error}"
        row["execution_accuracy"] = error_msg.strip()

print(f"Total comparado (com sucesso em ambas): {total_evaluated}")
print(f"✔️ Matches: {execution_match}")
print(f"❌ Mismatches: {total_evaluated - execution_match}")
print(f"✅ Execution Accuracy: {execution_match / total_evaluated * 100:.2f}%")

In [ ]:
import sqlite3
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase


class ExecutionAccuracyMetric(BaseMetric):
    def __init__(self):
      self.name = "Execution Accuracy"
      self.is_higher_better = True
      self.threshold = 1.0

    def execute_and_fetch(self, db_path, sql_command):
      conn = sqlite3.connect(db_path)
      cursor = conn.cursor()

      cursor.execute(sql_command)
      results = cursor.fetchall()

      conn.close()
      return results

    def measure(self, test_case: LLMTestCase) -> float:
      additional_metadata = test_case.additional_metadata
      self.error = None

      db_id = additional_metadata["db_id"]
      self.db_path = os.path.join(
        "spider/spider_data/database", db_id, f"{db_id}.sqlite")

      try:
        # Executa o SQL gold (esperado)
        pred_result = self.execute_and_fetch(self.db_path, test_case.actual_output)
      except Exception as e:
        self.error = f"[Error] Failed executing predicted SQL: {e}"

      try:
        gold_result = self.execute_and_fetch(self.db_path, test_case.expected_output)
      except Exception as e:
        self.error = f"[Error] Failed executing gold SQL: {e}"

      # Normaliza resultados (ordem não importa)
      if not self.error:
        pred_set = set(pred_result)
        gold_set = set(gold_result)
        self.score = int(pred_set == gold_set)
      else:
        self.score = 0

      self.success = self.score >= self.threshold

      return self.score

    def is_successful(self):
      return self.success

    @property
    def __name__(self):
      return "AccuracyMetric (NLP-TP-04)"


In [ ]:
metric = ExecutionAccuracyMetric()
test_case = LLMTestCase(
  input="List the name, born state and age of the heads of departments ordered by age.",
  # actual_output="SSELECT born_state ,  age FROM head ORDER BY age;",  # SQL gerado pelo modelo
  actual_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gerado pelo modelo
  expected_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gold
  additional_metadata={"db_id": "department_management"}
)
score = metric.measure(test_case)
print("Execution Accuracy:", score)

Execution Accuracy: 1


In [ ]:
from deepeval import assert_test, evaluate
from deepeval.evaluate.configs import AsyncConfig
from deepeval.test_case import LLMTestCase


metric = ExecutionAccuracyMetric()

test_cases = [
  LLMTestCase(
    input="List the name, born state and age of the heads of departments ordered by age.",
    actual_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gerado pelo modelo
    expected_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gold
    additional_metadata={"db_id": "department_management"}
  ),
  LLMTestCase(
    input="List the name, born state and age of the heads of departments ordered by age.",
    actual_output="SSSELECT born_state ,  age FROM head ORDER BY age;",  # SQL gerado pelo modelo
    expected_output="SELECT name ,  born_state ,  age FROM head ORDER BY age;",  # SQL gold
    additional_metadata={"db_id": "department_management"}
  )
]
result = evaluate(
  test_cases=test_cases,
  metrics=[metric],
  async_config=AsyncConfig(run_async=False)
)

✨ You're running DeepEval's latest AccuracyMetric (NLP-TP-04) Metric! (using None, strict=False, 
async_mode=False)...

Output()



Metrics Summary

  - ✅ AccuracyMetric (NLP-TP-04) (score: 1.0, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: None)

For test case:

  - input: List the name, born state and age of the heads of departments ordered by age.
  - actual output: SELECT name ,  born_state ,  age FROM head ORDER BY age;
  - expected output: SELECT name ,  born_state ,  age FROM head ORDER BY age;
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

AccuracyMetric (NLP-TP-04): 50.00% pass rate




Metrics Summary

  - ❌ AccuracyMetric (NLP-TP-04) (score: None, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: [Error] Failed executing predicted SQL: near "SSSELECT": syntax error)

For test case:

  - input: List the name, born state and age of the heads of departments ordered by age.
  - actual output: SSSELECT born_state ,  age FROM head ORDER BY age;
  - expected output: SELECT name ,  born_state ,  age FROM head ORDER BY age;
  - 

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

In [ ]:
import pandas as pd
from deepeval.dataset import EvaluationDataset
from deepeval.test_case import LLMTestCase


df = pd.read_csv("/content/spider_execution_accuracy_results.csv")
dataset = EvaluationDataset()
metric = ExecutionAccuracyMetric()

for _, row in df.iterrows():
  test_case = LLMTestCase(
    input=row.question,
    actual_output=row.predicted_sql,
    expected_output=row.gold_sql,
    additional_metadata={"db_id": row.db_id}
  )
  dataset.add_test_case(test_case)

result = evaluate(
  dataset,
  metrics=[metric],
  async_config=AsyncConfig(run_async=False)
)
# https://deepeval.com/docs/evaluation-datasets#without-pytest

✨ You're running DeepEval's latest AccuracyMetric (NLP-TP-04) Metric! (using None, strict=False, 
async_mode=False)...

Output()

A saída de streaming foi truncada nas últimas 5000 linhas.


Metrics Summary

  - ✅ AccuracyMetric (NLP-TP-04) (score: 1.0, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: None)

For test case:

  - input: Please show the name of the conductor that has conducted orchestras founded after 2008.
  - actual output: SELECT c.Name
FROM conductor c
JOIN orchestra o ON c.Conductor_ID = o.Conductor_ID
WHERE o.Year_of_Founded > 2008;
  - expected output: SELECT T1.Name FROM conductor AS T1 JOIN orchestra AS T2 ON T1.Conductor_ID  =  T2.Conductor_ID WHERE Year_of_Founded  >  2008
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

AccuracyMetric (NLP-TP-04): 42.94% pass rate




Metrics Summary

  - ✅ AccuracyMetric (NLP-TP-04) (score: 1.0, threshold: 1.0, strict: False, evaluation model: None, reason: None, error: None)

For test case:

  - input: What are the names of conductors who have conducted orchestras founded after the year 2008?
  - act

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

In [ ]:
# https://www.datacamp.com/tutorial/deepeval